<a href="https://colab.research.google.com/github/unburied/DS-Unit-2-Linear-Models/blob/master/Doing_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Get/Split/Model(basic)


In [0]:
import pandas as pd
import numpy as np

In [0]:
WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/nyc/nyc-rent-2016.csv'

df = pd.read_csv(WEB)

In [3]:
#View Columns
df.head().transpose()

,0,1,2,3,4
bathrooms,1.5,1,1,1,1
bedrooms,3,2,1,1,4
created,2016-06-24 07:54:24,2016-06-12 12:19:27,2016-04-17 03:26:41,2016-04-18 02:22:02,2016-04-28 01:32:41
description,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,,"Top Top West Village location, beautiful Pre-w...",Building Amenities - Garage - Garden - fitness...,Beautifully renovated 3 bedroom flex 4 bedroom...
display_address,Metropolitan Avenue,Columbus Avenue,W 13 Street,East 49th Street,West 143rd Street
latitude,40.7145,40.7947,40.7388,40.7539,40.8241
longitude,-73.9425,-73.9667,-74.0018,-73.9677,-73.9493
price,3000,5465,2850,3275,3350
street_address,792 Metropolitan Avenue,808 Columbus Avenue,241 W 13 Street,333 East 49th Street,500 West 143rd Street
interest_level,medium,low,high,low,low


In [27]:
#convert time string object to datetime object and set as index
df['created'] = pd.to_datetime(df.created, infer_datetime_format= True)

df = df.set_index('created')
df.head()

,bathrooms,bedrooms,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,exclusive,terrace,loft,garden_patio,common_outdoor_space,wheelchair_access
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-12 12:19:27,1.0,2,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-17 03:26:41,1.0,1,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-18 02:22:02,1.0,1,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-28 01:32:41,1.0,4,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
#Split Data Based on date
train = df[df.index.month < 6]
test = df[df.index.month == 6]

train.shape, test.shape

((31515, 33), (16785, 33))

In [31]:
#extract target values from df
y_train = train.pop('price')
y_test = test.pop('price')

y_train.shape, y_test.shape

((31515,), (16785,))

In [0]:
#get a baseline model
baseline = np.full_like(y_test, y_train.mean())

###Baseline

In [33]:
#get metric to test baseline 
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, baseline)

1052.5193327375632

###Upgrade Baseline Model


In [0]:
#import and instantiate model
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [0]:
#select features to model
#here we will start with all numerical features and exclude any objects
features = list(train.dtypes[train.dtypes != 'object'].index)

X_train = train[features]
X_test = test[features]

In [0]:
#Train model and store restults

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

###Upgraded Baseline

In [40]:
better_baseline = mean_absolute_error(y_test, y_pred)
better_baseline

618.6796864572029

###Feature Selection & Engineering
  

In [51]:
#Iterate over features to determine which have the highest impact

impact = {}

for feature in features:
  model.fit(train[[feature]], y_train)
  y_pred = model.predict(test[[feature]])
  impact[feature] = mean_absolute_error(y_test, y_pred)
                
print(impact)

{'bathrooms': 826.8186291735686, 'bedrooms': 865.7153554398519, 'latitude': 1048.5914341618604, 'longitude': 989.0487004130522, 'elevator': 1021.1914238867432, 'cats_allowed': 1049.6548133279234, 'hardwood_floors': 1050.474902267552, 'dogs_allowed': 1049.2944534089013, 'doorman': 1002.990620652015, 'dishwasher': 1026.0537865120189, 'no_fee': 1044.9120965283466, 'laundry_in_building': 1052.8562017778665, 'fitness_center': 1020.8674997057973, 'pre-war': 1052.1651020895426, 'laundry_in_unit': 1028.7246629448905, 'roof_deck': 1044.951424200772, 'outdoor_space': 1038.9376029999073, 'dining_room': 1029.926584738313, 'high_speed_internet': 1047.8368618041293, 'balcony': 1040.796298235273, 'swimming_pool': 1046.0479246061477, 'new_construction': 1052.4032354966685, 'exclusive': 1052.262266759364, 'terrace': 1042.9198257172632, 'loft': 1051.8527496972877, 'garden_patio': 1046.3402013438135, 'common_outdoor_space': 1051.7843743981514, 'wheelchair_access': 1051.120211011886}


In [54]:
#hard to see so throw into df

impact_table = pd.DataFrame(impact, index = len(impact))

impact

TypeError: ignored